In [93]:
import os
import pandas as pd
#from pytube import YouTube
from moviepy.editor import *
import librosa
import soundfile as sf
import numpy as np
import shutil

In [94]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [95]:
# 경로 정의
DATA_IN_PATH = 'D:/PBL 스터디/PBL 실습/audio&data/'
PATH_DATA = 'D:/PBL 스터디/PBL 실습/'
INPUT_DATA = 'clova_result.csv'

test_data = pd.read_csv(PATH_DATA+'clova_result.csv', header = 0)
pred_df = pd.DataFrame({'sentence':test_data['sentence'], 'start':test_data['start'], 'end':test_data['end'],'predict':test_data['results']})
pred_df

,sentence,start,end,predict
0,이번 신제품 출시에 대한 시장의 반응은 어때. 미친 개 같아.,980,8590,0.937683
1,시장의 반응이 차가워. 아 그런 젓 같은 일이,9880,15740,0.961118
2,우린 이제 야근의 연속이겠다. 판매량이 지난번 제품보다는 좋다고는 하는데,17430,26290,0.043407
3,모르겠다. 뭐가 문제인 걸까. 우리가 엿 같이 한 탓이지.,26480,35740,0.645844
4,야 야 진정해. 다시 분석해 보자.,37180,40240,0.097138


In [96]:
start = []
end = []
result = []

for i in range(len(pred_df)):
    start.append(int(pred_df['start'][i]*44.1))
    end.append(int(pred_df['end'][i]*44.1))
    
    if pred_df['predict'][i] >= 0.7:
        result.append(1)
    else:
        result.append(0)



In [97]:
result

[1, 1, 0, 0, 0]

In [98]:
# # 단위: ms (1000분의 1초)
# start = [980, 9880, 17380, 26480, 37180]
# for i in range(0, len(start)):
#     start[i] = int(start[i]*44.1)
# end = [8590, 15740, 26290, 35740, 40190]
# for i in range(0, len(end)):
#     end[i] = int(end[i] * 44.1)
# result = [1, 1, 0, 1, 0]
# start

In [99]:
# def make_quiet_wav(wav, y, output_name):
#     quiet, _ = librosa.load(DATA_IN_PATH+'quiet.wav', sr = 44100)
#     data, _ = librosa.load(wav, sr=44100)
#     length = len(y)
#     for i in range(1, length):
#         tmp = int(len(data)*(i)/length)
#         t_1 = y[i-1] # 이전 구간
#         t = y[i] # 현재구간
#         if (t_1 == 0) and (t == 1):
#             if tmp > 22000 :
#                 data[tmp-22000:tmp] = quiet[:22000] * 1.0
#             else:
#                 data[:tmp] = quiet[:tmp] * 1.0  #a[2]-a[1]
#     sf.write(output_name, data, 44100)

In [100]:
def make_quiet_wav(wav, result, start, end, output_name):
    quiet, _ = librosa.load(DATA_IN_PATH+'quiet.wav', sr = 44100) # 무음 파일 로드
    data, _ = librosa.load(wav, sr=44100) # 변환할 음성 파일 로드
    # data -> samplerate로 나누면 초(s) 단위
    
    length = len(result) # 단위 개수
    lenq = len(quiet) #694575
    
    for i in range(0, length):
        tmp = int(end[i] - start[i])
        r = result[i]
        print("for문 : [{}] {} : {}".format(r, start[i]/44100, end[i]/44100))
        if (r == 1):
            if tmp > lenq:
                data[end[i]-lenq:end[i]] = quiet[:]
                print("quiet : {}:{}".format((end[i]-lenq)/44100, end[i]/44100))
            else:
                data[start[i]:end[i]] = quiet[:tmp] * 1.0  
                print("quiet : {}:{}".format(start[i]/44100, end[i]/44100))
    sf.write(output_name, data, 44100)

In [101]:
def main(video, idd=0):
    data_output = DATA_IN_PATH + str(idd) + "\\"
    print(bcolors.OKGREEN + '\nplease wait! \n' + bcolors.ENDC)

    # 파일 복사하기 (사본 생성)
    shutil.copy2(DATA_IN_PATH+video, data_output+"copied_"+video)
    print(bcolors.OKGREEN + '\nMake copy of video ! \n' + bcolors.ENDC)

    videoclip = VideoFileClip(data_output+"copied_"+video)
    audioclip = videoclip.audio
    audioclip.write_audiofile(data_output+"copy.wav") # 음성 wav 추출하기
    print(bcolors.OKGREEN + '\nChage mp4 to wav ! \n' + bcolors.ENDC)
    
    y =  [0,1,0,0,0,1,0,0,1,0]
    #      0 1 2 3 4 5 6 7 8 9
    # 0 3~4 6~7

    # 0    1    2  3
    # a   1.0  3.0 아메리카노
    make_quiet_wav(data_output+"copy.wav", result, start, end, data_output+"{}_result_{}.wav".format(idd, video))
    videoclip = videoclip.set_audio(AudioFileClip(data_output+"{}_result_{}.wav".format(idd, video)))
    videoclip = videoclip.subclip(0, 42)
    videoclip.write_videofile(data_output+"{}_result_{}".format(idd, video))

    print(bcolors.OKGREEN + '\nNow you can check!!! ' + bcolors.ENDC)

    # 데이터 길이 10개, 0->1로 바뀌는 구간 총 5번. 실제 묵음 처리 5번
    # 데이터 길이 5개, 0->1로 바뀌는 구간 2번. 실제 묵음 동일.
    # 영상 길이와 데이터 길이에 따라 구간 변경?
    # 영상 길이에 따라 간격은 달라지는 것 같음.
    # 앞 팀에게 데이터 요청시 start time 과 endtime 분배 요청하기

if __name__ == '__main__':
    video= "test_audio.mp4"
    #video= "small_test_audio.mp4"
    idd = int(input('ID: '))
    data_output = DATA_IN_PATH + str(idd) + "\\"
    try:
        if not os.path.exists(data_output):
            os.makedirs(data_output)
    except OSError:
        print('Error: Creating directory. ' + data_output)
        quit()

    main(video, idd)
    print(bcolors.OKGREEN + '\nDone !!! \n' + bcolors.ENDC)


ID: 32

please wait! 


Make copy of video ! 

MoviePy - Writing audio in D:/PBL 스터디/PBL 실습/audio&data/32\copy.wav


MoviePy - Done.

Chage mp4 to wav ! 

for문 : [1] 0.98 : 8.59
quiet : 0.98:8.59
for문 : [1] 9.88 : 15.74
quiet : 9.88:15.74
for문 : [0] 17.43 : 26.29
for문 : [0] 26.48 : 35.74
for문 : [0] 37.18 : 40.24
Moviepy - Building video D:/PBL 스터디/PBL 실습/audio&data/32\32_result_test_audio.mp4.
MoviePy - Writing audio in 32_result_test_audioTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video D:/PBL 스터디/PBL 실습/audio&data/32\32_result_test_audio.mp4



Moviepy - Done !
Moviepy - video ready D:/PBL 스터디/PBL 실습/audio&data/32\32_result_test_audio.mp4

Now you can check!!! 

Done !!! 

